# IBM WATSON TONE ANALYZER 

## Getting Started with IBM Watson 

In [ ]:
!pip install ibm_watson 

## Getting Started with Python Package to work with Telegram

In [ ]:
!pip install telethon

## Credentials Setup 

In [ ]:
#@title Private IBM Watson Credentials 
apikey = 'PQH09s7Ui-FFNPrv_QhIHHqC2SklFk-c-btGSZCEEdBk'
url = 'https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/0fcd3e3c-4c90-4b6b-971c-f13f8eb1ccd2'

## Watson Libraries Import 


In [ ]:
# https://betterprogramming.pub/how-to-get-data-from-telegram-82af55268a4b 
# https://www.youtube.com/watch?v=1LHLiBhASSA
from ibm_watson import ToneAnalyzerV3 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

## Telegram Libraries Import 

In [ ]:
import configparser
import json
import asyncio
from datetime import date, datetime
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch
from telethon.tl.functions.messages import (GetHistoryRequest)
from telethon.tl.types import (PeerChannel)

## Tone Analyzer Setup 

In [ ]:
authenticator = IAMAuthenticator(apikey)
ta = ToneAnalyzerV3(version='2017-09-21', authenticator=authenticator)
ta.set_service_url(url)

## Config Telegram 

In [ ]:
config = configparser.ConfigParser()
config.read("config.ini")

# Setting configuration values
api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']

api_hash = str(api_hash)

phone = config['Telegram']['phone']
username = config['Telegram']['username']

In [ ]:
# Create the client and connect
api_id = 3177433
api_hash = 'e3efc871e777bfee6140b1a9a8af5ed5'
client = TelegramClient('Tele_Tone_Bot', api_id, api_hash)
client.start(bot_token='1729758498:AAF0fRnEQavTgIGMAzRMog4PG6oca9NWvJ4')
print(f'Bot launched on @{client.get_me()}')

Bot launched on @<coroutine object UserMethods.get_me at 0x7ff9a29e64d0>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: coroutine 'AuthMethods._start' was never awaited
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: coroutine 'UserMethods.get_me' was never awaited
  


## Get User Input

In [ ]:
async def main(phone):
    await client.start()
    print("Client Created")
    # Ensure you're authorized
    if await client.is_user_authorized() == False:
        await client.send_code_request(phone)
        try:
            await client.sign_in(phone, input('Enter the code: '))
        except SessionPasswordNeededError:
            await client.sign_in(password=input('Password: '))

    me = await client.get_me()

    user_input_channel = input('enter entity(telegram URL or entity id):')

    if user_input_channel.isdigit():
        entity = PeerChannel(int(user_input_channel))
    else:
        entity = user_input_channel

    my_channel = await client.get_entity(entity)

    offset_id = 0
    limit = 100
    all_messages = []
    total_messages = 0
    total_count_limit = 0

    while True:
        print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
        history = await client(GetHistoryRequest(
            peer=my_channel,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0
        ))
        if not history.messages:
            break
        messages = history.messages
        for message in messages:
            all_messages.append(message.to_dict())
        offset_id = messages[len(messages) - 1].id
        total_messages = len(all_messages)
        if total_count_limit != 0 and total_messages >= total_count_limit:
            break

    with open('channel_messages.json', 'w') as outfile:
        json.dump(all_messages, outfile, cls=DateTimeEncoder)

## Text Input / Extract Text 

In [ ]:
text = 'In another life I would be your girl We would keep all our promises Be us against the world' 

## Tone Generation

In [ ]:
res = ta.tone(text).get_result()
Tone_Style = res['document_tone']['tones'][0]['tone_name']
print(Tone_Style)

Confident


In [ ]:
text_new = text.replace(' ','')

In [ ]:
res = text.split()

In [ ]:
res_new = res[3:-1]

In [ ]:
res_new 

In [ ]:
new_text = ''
for word in res_new:
  word = word + ' '
  new_text += word
new_text

'I would be your girl We would keep all our promises Be us against the '

In [ ]:
res = ta.tone(new_text).get_result()
print(res)

{'document_tone': {'tones': [{'score': 0.727798, 'tone_id': 'confident', 'tone_name': 'Confident'}]}}
